<a href="https://colab.research.google.com/github/HeedfulMoss/DEEP_ML_Project/blob/main/Training8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install python-docx
!pip install datasets peft transformers wandb
!pip install torchinfo
!pip install datasets
from google.colab import drive
drive.mount('/content/drive')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 82.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import numpy as np
import torch
from torch import nn
from datasets import load_dataset
from transformers import (
    LongformerTokenizer,
    LongformerForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)
from peft import LoraConfig, get_peft_model
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score, f1_score
import wandb
from torchinfo import summary


# --- Initialize WandB ---
wandb.init(project="longformer-icd9-classification")

# --- Constants ---
MODEL_NAME = "allenai/longformer-base-4096"
DATA_PATH = '/content/drive/MyDrive/DEEP_ML_Project-main/data/preprocessed/summary_results.csv'
OUTPUT_DIR = "./longformer_icd9_define"
MAX_LENGTH = 512 # Initialize Longformer tokenizer & model

# --- Load Data ---
#dataset = load_dataset("csv", data_files=DATA_PATH, split="train")
# Replaces manual slicing with a single train_test_split
dataset = load_dataset("csv", data_files=DATA_PATH)["train"].train_test_split(test_size=0.1, seed=42)
train_dataset = dataset["train"]
val_dataset   = dataset["test"]


# Define ICD-9 columns and mapping
icd9_columns = [col for col in train_dataset.column_names if col.startswith("ICD9_")]
# --- ICD-9 Code to Meaning Mapping ---
ICD9_MAPPING = {
    "ICD9_4019": "Unspecified essential hypertension",
    "ICD9_4280": "Congestive heart failure, unspecified",
    "ICD9_42731": "Atrial fibrillation",
    "ICD9_41401": "Coronary atherosclerosis of native coronary artery",
    "ICD9_5849": "Acute kidney failure, unspecified",
    "ICD9_25000": "Diabetes mellitus without mention of complication, type II or unspecified type",
    "ICD9_2724": "Other and unspecified hyperlipidemia",
    "ICD9_51881": "Acute respiratory failure",
    "ICD9_5990": "Urinary tract infection, site not specified",
    "ICD9_53081": "Esophageal reflux",
    "ICD9_2720": "Pure hypercholesterolemia",
    "ICD9_V053": "Need for prophylactic vaccination and inoculation against viral hepatitis",
    "ICD9_V290": "Observation for suspected infectious condition",
    "ICD9_2859": "Anemia, unspecified",
    "ICD9_2449": "Unspecified acquired hypothyroidism",
    "ICD9_486": "Pneumonia, organism unspecified",
    "ICD9_2851": "Acute posthemorrhagic anemia",
    "ICD9_2762": "Acidosis",
    "ICD9_496": "Chronic airway obstruction, not elsewhere classified",
    "ICD9_99592": "Severe sepsis"
}

# Convert one-hot columns to label list
def encode_labels(example):
    example['labels'] = [example[c] for c in icd9_columns]
    return example

# Convert ICD-9 binary flags to list of textual meanings
def get_text_labels(batch):
    labels = []
    for idx in range(len(batch[icd9_columns[0]])):
        active_labels = [
            ICD9_MAPPING[col] for col in icd9_columns
            if batch[col][idx] == 1
        ]
        labels.append(active_labels)
    return {"text_labels": labels}

def encode_text_labels(batch):
    return {"labels": mlb.transform(batch["text_labels"]).tolist()}

# Replaces manual slicing with a single train_test_split
#dataset = dataset.map(get_text_labels, batched=True)

train_dataset = train_dataset.map(get_text_labels, batched=True)
val_dataset   = val_dataset.map(get_text_labels, batched=True)

# Get all possible ICD-9 meanings
all_possible_labels = list(ICD9_MAPPING.values())

# Initialize MultiLabelBinarizer
mlb = MultiLabelBinarizer(classes=all_possible_labels)
mlb.fit([all_possible_labels])  # Dummy fit to include all labels

train_dataset = train_dataset.map(encode_text_labels, batched=True)
val_dataset   = val_dataset.map(encode_text_labels, batched=True)

# 5) if you no longer need the original ICD9_* and text_labels columns:
to_drop = icd9_columns + ["text_labels"]
train_ds = train_dataset.remove_columns(to_drop)
val_ds   = val_dataset.remove_columns(to_drop)

# --- Tokenize Text (Clinical Notes) ---
tokenizer = LongformerTokenizer.from_pretrained(MODEL_NAME)

def tokenize(batch):
    return tokenizer(
        batch["summary_snippet_clean"],
        padding="max_length",
        truncation=True,
        max_length=MAX_LENGTH,
        return_tensors="pt"
    )

train_dataset = train_dataset.map(tokenize, batched=True, batch_size=8)
val_dataset = val_dataset.map(tokenize, batched=True, batch_size=8)

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: goldwolf228 (goldwolf228-university-of-north-carolina-at-charlotte) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Generating train split: 0 examples [00:00, ? examples/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

Map:   0%|          | 0/41934 [00:00<?, ? examples/s]

Map:   0%|          | 0/4660 [00:00<?, ? examples/s]

In [ ]:
# --- Class Weighting ---
all_labels = np.array(train_dataset["labels"])
class_weights = compute_class_weight(
    "balanced",
    classes=np.arange(len(all_possible_labels)),
    y=all_labels.argmax(axis=1) if len(all_possible_labels) > 1 else all_labels.ravel()
)
class_weights = torch.tensor(class_weights, dtype=torch.float32)

# --- Model Setup ---
model = LongformerForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=len(all_possible_labels),
    problem_type="multi_label_classification"
)

# Print model summary with torchinfo
batch_size = 2
summary(model,
        input_size=(batch_size, MAX_LENGTH),
        dtypes=[torch.long],
        verbose=1)

# Print total parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")
wandb.log({"total_parameters": total_params, "trainable_parameters": trainable_params})

# Switch from fixed-length padding to dynamic with DataCollator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# LoRA for efficiency
lora_config = LoraConfig(
    task_type="SEQ_CLS",
    r=8,
    lora_alpha=16,
    target_modules=["query", "value"],
    lora_dropout=0.05,
    bias="none"
)
model = get_peft_model(model, lora_config)

# train_dataset["labels"] is a list of 0/1 lists, one per sample
binary_labels = np.stack(train_dataset["labels"])  # shape (n_samples, n_classes)

# count positives and negatives per class
pos_counts = binary_labels.sum(axis=0)
neg_counts = binary_labels.shape[0] - pos_counts

# weight = neg / pos  (you can also normalize further if you like)
class_weights = neg_counts / (pos_counts + 1e-6)

# to torch tensor
class_weights = torch.tensor(class_weights, dtype=torch.float32)

# --- Metrics Calculation ---
def compute_metrics(p):
    probs = torch.sigmoid(torch.tensor(p.predictions)).numpy()
    preds = (probs > 0.5).astype(int)  # Threshold at 0.5
    labels = p.label_ids

    # Calculate metrics
    accuracy = accuracy_score(labels, preds)
    f1_micro = f1_score(labels, preds, average='micro')
    f1_macro = f1_score(labels, preds, average='macro')

    return {
        'accuracy': accuracy, # label-wise accuracy
        'f1_micro': f1_micro,
        'f1_macro': f1_macro
    }

# --- Custom Trainer (Weighted Loss) ---
class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        # pop off labels and move to float
        labels = inputs.pop("labels").to(torch.float32)
        # forward pass
        outputs = model(**inputs)
        logits = outputs.logits

        # compute weighted BCE loss
        weights = class_weights.to(logits.device)[None, :].expand(logits.size(0), -1)
        loss_fct = nn.BCEWithLogitsLoss(weight=weights, reduction="mean")
        loss = loss_fct(logits, labels)

        if return_outputs:
            return loss, outputs
        return loss

# --- Training ---
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    learning_rate=2e-5,
    eval_strategy="steps",
    eval_steps=500,
    save_strategy="steps",
    save_steps=1000,
    logging_steps=100,
    fp16=True,
    report_to="wandb",
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
)

trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

# Train the model
train_results = trainer.train()

# Save metrics
trainer.save_metrics("train", train_results.metrics)
val_metrics = trainer.evaluate()
trainer.save_metrics("eval", val_metrics)

# Print final metrics
print("\nTraining completed!")
print(f"Training loss: {train_results.metrics['train_loss']:.4f}")
print(f"Validation loss: {val_metrics['eval_loss']:.4f}")
print(f"Validation accuracy: {val_metrics['eval_accuracy']:.4f}")
print(f"Validation F1 (macro): {val_metrics['eval_f1_macro']:.4f}")

# Save model and tokenizer
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

# Finish WandB run
wandb.finish()

Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Layer (type:depth-idx)                                       Output Shape              Param #
LongformerForSequenceClassification                          [2, 20]                   --
├─LongformerModel: 1-1                                       [2, 512, 768]             --
│    └─LongformerEmbeddings: 2-1                             [2, 512, 768]             --
│    │    └─Embedding: 3-1                                   [2, 512, 768]             38,603,520
│    │    └─Embedding: 3-2                                   [2, 512, 768]             3,147,264
│    │    └─Embedding: 3-3                                   [2, 512, 768]             768
│    │    └─LayerNorm: 3-4                                   [2, 512, 768]             1,536
│    │    └─Dropout: 3-5                                     [2, 512, 768]             --
│    └─LongformerEncoder: 2-2                                [2, 512, 768]             --
│    │    └─ModuleList: 3-6                                  --             

<ipython-input-4-162821ab4bb7>:112: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `WeightedTrainer.__init__`. Use `processing_class` instead.
  trainer = WeightedTrainer(
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss,Validation Loss,Accuracy,F1 Micro,F1 Macro
500,2.200600,2.188742,0.030901,0.058625,0.082459
1000,2.171000,2.111463,0.032833,0.063302,0.085154
1500,2.107400,2.054256,0.039270,0.164415,0.126405
2000,2.030400,2.029166,0.040987,0.202432,0.144413
2500,2.057400,2.008617,0.048712,0.255754,0.162158
3000,2.016700,1.998835,0.048283,0.284206,0.193339
3500,2.035500,1.981947,0.048712,0.297903,0.201201
4000,1.980400,1.969683,0.050000,0.281861,0.190596
4500,2.014400,1.956196,0.048927,0.285973,0.198422
5000,2.011300,1.951722,0.054506,0.328879,0.229543



Training completed!
Training loss: 2.0102
Validation loss: 1.8559
Validation accuracy: 0.0706
Validation F1 (macro): 0.3344


eval/accuracy,▁▁▂▃▄▄▄▄▄▅▆▆▅▆▆▆▆▆▇▇███████
eval/f1_macro,▁▁▂▃▃▄▄▄▄▅▅▅▆▆▆▇▆▇▇████████
eval/f1_micro,▁▁▃▄▅▅▆▅▅▆▇▇▇▇▇▇▇██████████
eval/loss,█▆▅▅▄▄▄▃▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁
eval/runtime,▄▂▄▅▅█▄▄▃▃▃▁▂▂▂▂▅▆▅▃▅▄▅▆▄▄▄
eval/samples_per_second,▅▇▅▄▄▁▅▅▆▆▆█▇▇▇▇▄▃▄▆▄▅▄▃▅▅▅
eval/steps_per_second,▅▇▅▄▄▁▅▅▅▆▆█▇▇▇▇▄▃▄▆▄▅▄▃▅▅▄
total_parameters,▁▁
train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▅▅▅▅▅▅▅▅▅▅▆▆▇▇▇▇▇▇▇▇██
train/global_step,▁▁▁▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇█
train/grad_norm,▂▂▂▂▄▁▂▂▂▆▃▃▃▂▃▃▄█▂▂▄▃▂▆▄▄▄▇▂▄▄▃▄▂▄▅▃▄▂▄


In [ ]:
from transformers import LongformerForSequenceClassification, LongformerTokenizer

model_path = '/content/drive/MyDrive/DEEP_ML_Project-main/models/longformer_icd9_define'  # Your output directory
NUM_LABELS = 20  # Should match number of ICD-9 codes you trained with

model = LongformerForSequenceClassification.from_pretrained(
    model_path,
    num_labels=NUM_LABELS,
    problem_type="multi_label_classification"
)
tokenizer = LongformerTokenizer.from_pretrained(model_path)

Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def preprocess_text(text):
    # Tokenize with same parameters as training
    inputs = tokenizer(
        text,
        padding="max_length",
        truncation=True,
        max_length=4096,
        return_tensors="pt"
    )
    return inputs

In [ ]:
def predict_icd_codes(text, threshold=0.20):
    # Preprocess
    inputs = preprocess_text(text)

    # Predict
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probs = torch.sigmoid(logits)  # Convert to probabilities

    # Apply threshold
    predictions = (probs > threshold).int().squeeze().tolist()

    # Get predicted ICD codes
    predicted_indices = [i for i, pred in enumerate(predictions) if pred == 1]
    predicted_codes = [icd9_columns[i] for i in predicted_indices]
    predicted_descriptions = [ICD9_MAPPING[code] for code in predicted_codes]

    return {
        "raw_probabilities": probs.squeeze().tolist(),
        "predicted_icd9_codes": predicted_codes,
        "predicted_descriptions": predicted_descriptions
    }


In [ ]:
sample_note = """
"Admission Date: [DATE] Discharge Date: [DATE]
Date of Birth: [DATE] Sex: F
Service: [DEIDENTIFIED]
Allergies:
Haldol / Darvon / Keppra
Attending:[NAME]
Chief Complaint:
Black stools
Major Surgical or Invasive Procedure:
s/p EGD [DATE], blood transfusion
History of Present Illness:
This is a 62 y/o female with PMH significant for HCV/ESLD, h/o
GIB, recently admitted from [DEIDENTIFIED] for a signficant LGIB
s/p TIPS, p/w black stools, lethargy, and confusion. Pt noted to
have a HCT of 18 and was transfused 2 Units PRBC's at 0530 on
[DEIDENTIFIED].
Most recent hospitilization was c/b respiratory failure [DEIDENTIFIED]
pulmonary edema and nosocomial PNA, she was intubated for 10
days and completed a course of antibiotics for 12 days as well
as diuresis with lasix. Her LGIB was thought to be [DEIDENTIFIED] rectal
hemorrhoids and required [NAME] [NAME] at some point during her
course. She was discharged to rehab on [DEIDENTIFIED] in stable condition
with a Hct of 30.
In the ED, initial VS were T 97.4, 68, 99/60, 14, 100% RA. An
abdominal u/s was performed and demonstrated a patent TIPS. Head
CT revealed no acute intracranial hemorrhage, edema, or mass
effect. HCT was 24.4. Foley with 400 cc of UO.
She was given vitamin K 10 Units SC, 40 mg IV protonix,
lactulose 30 ml, and 2 U FFP. The hepatology service was made
aware and plan to scope her tomorrow. She was admitted to the
MICU for further management.
Upon admit she is encephalopathic and moaning. Unable to obtain
history.
Past Medical History:
1) iron deficiency anemia
2) GI bleed - presumed secondary to hemorrhoids
3) Sigmoid diverticulosis
4) Schatzki's ring
5) Duoenal polyps and duodenitis
6) MGUS
7) ?etoh/ HCV cirrhosis followed by Dr. [NAME] (vl 9k in [DEIDENTIFIED])
8) psychotic disorder
9) remote polysubstance abuse - etoh, cocaine, marijuana
10) COPD
11) compex partial seizures
Social History:
History of tobacco and EtOH abuse. She is originally from
[STATE], and changed her name when she became a practicing
Muslim. She worked as an administrative assistant when she was
younger, but is now on SSDI (for schizophrenia and seizure
disorder, per pt, both now quiescent).
Family History:
Mother: asthma, grandmother with diabetes, HTN. No family
history of liver disease or bleeding disorders. Great aunt with
epilepsy.
Physical Exam:
VS: T97.2 142/73 84 15 96/RA
General: Verbalizing with intermittently sensible answers, A&Ox2
HEENT: MMM, sclera icteric, not following commands for neuro
exam
Chest: CTAB anteriorly and in axilla, cannot roll to listen
posteriorly
CV: RRR difficult to ausculate m/g/r as patient continues to
talk
Abd: (+) BS, soft, NT/ND
Ext: 2+ pitting edema, strong pulses; RUE hand 2+ edema
Pertinent Results:
[DATE] 12:00PM PT-17.7* PTT-46.5* INR(PT)-1.6*
[DATE] 12:00PM WBC-8.9 RBC-2.68* HGB-8.4* HCT-24.4* MCV-91
MCH-31.4 MCHC-34.6 RDW-19.8* NEUTS-79* BANDS-4 LYMPHS-11*
MONOS-6 EOS-0 BASOS-0 ATYPS-0 METAS-0 MYELOS-0
[DATE] 12:00PM HYPOCHROM-NORMAL ANISOCYT-2+ POIKILOCY-NORMAL
MACROCYT-NORMAL MICROCYT-NORMAL POLYCHROM-NORMAL
[DATE] 12:00PM ALT(SGPT)-51* AST(SGOT)-109* ALK PHOS-72
AMYLASE-209* TOT BILI-5.7*
[DATE] 12:00PM AMMONIA-86*
[DATE] 12:00PM
GLUCOSE-99 UREA N-98* CREAT-1.9* SODIUM-145 POTASSIUM-6.0*
CHLORIDE-110* TOTAL CO2-23 ANION GAP-18 LACTATE-1.9 K+-6.7*
[DATE] 01:57PM K+-4.8
[DATE] 12:35PM URINE
BLOOD-NEG NITRITE-NEG PROTEIN-NEG GLUCOSE-NEG KETONE-NEG
BILIRUBIN-NEG UROBILNGN-NEG PH-7.0 LEUK-NEG
Brief Hospital Course:
1. Acute blood loss anemia secondary to GI hemorrhage- The
patient was admitted for melena, likely secondary to an upper GI
bleed. She received a total of 2 U PRBCs upon admission with
Lasix following the transfusion. She was started on Protonix IV
bid on admission. An abdominal u/s in the ED demonstrated a
patent TIPS. Her hematocrit remained stable overnight and she
was planned for endoscopy the following morning. She was quite
encephalopathic upon admission and refused to take any of her
medications, including lactulose and rifaximin. Due to the
encephalopathy and mental status, the patient was electively
intubated prior to the EGD on [DATE], without any
complications. An EGD on [DATE] showed portal gastropathy only
without any evidence of active bleeding. There was no evidence
of varices. She was extubated successfully the following day on
[DATE] and her respiratory status has been stable since on room
air. Since her EGD she was without further episodes of BRBPR or
melena. Her HCT was slowly downward trending and on [DATE]
she received one additional unit 1u PRBC with an appropriate HCT
increase to 28.9. Haptoglobin was also checked and found to be
<20, consistent with either slight hemolysis or could be
attributed to underlying liver disease. Otherwise, most likely
has subacute bleeding as a chronic issue secondary to
gastropathy or hemorrhoids. Upon discharge, was sent to an
extended care facility with instruction to transfuse for any HCT
less than 25, and to check a CBC every other day.
# HCV Cirrhosis with AMS - Known to have h/o hepatic
encephalopathy, was not tolerating lactulose/rifaximin so NGT
was placed. Given lactulose with improvement. She later
self-discontinued the NGT and subsequently tolerated oral
medications. She was continued on rifaximin & lactulose and her
mental status returned to baseline. Was also restarted on low
dose Lasix and Aldactone, which she tolerated well prior to
discharge.
# Hypernatrema - Thought to be [DEIDENTIFIED] decreased oral intake and
dehydration. Initially she was corrected with free water
boluses with her tube feeds. After removal of the NGT, she was
given IV D5W for correction given that she was unable to take in
enough oral hydration to correct it herself. Over several days,
she was slowly corrected and the day of discharge had a sodium
level of 140. Upon discharge, instructions were given to her
extended care facility for monitoring or her sodium levels, as
well as instructions for correction based on the current level.
# Acute renal failure with Chronic Renal Insufficiency -
Creatinine at recent baseline (1.7-2.0), and admitted with
creatinine 1.9. Unclear etiology, but originally thought to be
likely secondary to end stage liver disease. Increased BUN may
be secondary to GI Bleed. Also may be elevated from baseline
given free water deficit. Later improved to baseline after
treatment of free water deficits. Upon discharge, his
creatinine was 1.3. Extended care facility given instruction to
monitor creatinine and dose diuretics appropriately.
# COPD - Admitted at baseline without evidence of acute flair,
and clear pulmonary exam. Was intubated due to AMS, but was
later successfully extubated, as described above. The morning
of [DEIDENTIFIED], had acute onset wheezing, thought to be most likely
due to volume overload in the setting of increased free water
rather than acute flair. Was treated with IV Lasix and
albuterol nebulizers with symptomatic improvement. Once fluid
repletion was done more gently and home diuretics were
restarted, she had no further episodes of dyspnea.
Medications on Admission:
- Lansoprazole 30 mg daily
- Olanzapine 5 mg tid prn
- ?Beclomethasone Dipropionate 80 mcg as needed for SOB.
- Camphor-Menthol 0.5-0.5 % Lotion TID as needed for itching.
- Albuterol nebs q6 hrs
- Albuterol nebs q 1-2 hrs prn
- Ipratropium q6 hrs
- Lactulose 30 mL qid
- Rifaximin 400 mg tid
- ?Labetalol 100 mg [HOSPITAL]
- Haloperidol 0.5 mg tid
- ?Lasix 40 mg daily
- ?Spironolactone 50 mg daily
Discharge Medications:
1. Rifaximin 200 mg Tablet [HOSPITAL]: Two (2) Tablet PO TID (3 times a
day).
2. Lactulose 10 gram/15 mL Syrup [HOSPITAL]: Thirty (30) ML PO TID -
QID: You will need to titrate this medication to 4 BM's per day;
this medication is necessary to keep her mentally clear.
3. Nystatin 100,000 unit/mL Suspension [HOSPITAL]: Five (5) ML PO QID
(4 times a day) as needed.
4. Ipratropium Bromide 0.02 % Solution [HOSPITAL]: One (1) Nebulizer
Inhalation Q6H (every 6 hours).
5. Albuterol Sulfate 0.083 % (0.83 mg/mL) Solution [HOSPITAL]: One (1)
Nebulizer Inhalation Q6H (every 6 hours) as needed for shortness
of breath or wheezing.
6. Pantoprazole 40 mg Tablet, Delayed Release (E.C.) [HOSPITAL]: One
(1) Tablet, Delayed Release (E.C.) PO Q24H (every 24 hours).
7. Furosemide 20 mg Tablet [HOSPITAL]: One (1) Tablet PO DAILY (Daily):
Hold for SBP < 100 .
8. Spironolactone 25 mg Tablet [HOSPITAL]: Two (2) Tablet PO DAILY
(Daily): Hold for SBP < 100 or potassium > 5 .
9. Outpatient Occupational Therapy
Please evaluate and treat while at extended care facility.
10. Outpatient Physical Therapy
Please evaluate and treat while inpatient at facility. Given
patient's physical decompensation, exercise may need to be
primarily with upper extremities.
11. Outpatient Lab Work
Please check basic chemistry and CBC every other day and adhere
to the following guidelines:
***Contact the facility physician [NAME]***
1. If Hct < 25 or has dropped more than 5 points from prior
level, transfuse 1u PRBC and perform repeat Hct.
2. If Na (sodium) level is 143-145, please give 1L D5W at
100cc/hour. If Na (sodium) level is 146-148, please give 1.5L
D5W at 100cc/hour.
3. If Cr if > 1.8, hold lasix for the day and recheck lab the
following morning.
4. If potassium (K) > 4.8, hold spironolactone until potassium
is less than 4.5.
12. Ferrous Sulfate 325 mg (65 mg Iron) Tablet [NAME]: One (1)
Tablet PO DAILY (Daily).
Discharge Disposition:
Extended Care
Facility:
[HOSPITAL] [HOSPITAL] Hospital
Discharge Diagnosis:
Primary: Hepatitis C Cirrhosis, hemorrhoids
Secondary: COPD, iron defciency anemia, psychotic disorder,
diverticulosis, MGUS
Discharge Condition:
Hemodynamically stable and afebrile
Discharge Instructions:
You were admitted for black stools and increased confusion in
the setting of ongoing blood loss in your colon. You were
treated with blood transfusions and evaluated with an EGD. You
were also continued on your medications for your other ongoing
medical problems. Once improved, you were discharged back to
your extended care facility for further rehabilitation.
Please continue taking all medications as prescribed. You
living facility has been provided with a list of all your
current medications.
Please keep all medical appointments. You will also be treated
by the facility physician while you are living there.
Please return to the hospital or consult your facility physician
if you notice black stools, bright red blood in your bowel
movements, inability to eat enough food to maintain your weight,
increased confusion, bloody vomit or for any other symptom which
is concerning to you.
Followup Instructions:
Provider: [NAME] [NAME], MD Phone:[PHONE]
Date/Time:[DATE] 2:20"

"""

results = predict_icd_codes(sample_note)
print("Predicted ICD-9 Codes:", results["predicted_icd9_codes"])
print("Descriptions:", results["predicted_descriptions"])
print("All Probabilities:", list(zip(icd9_columns, results["raw_probabilities"])))

Predicted ICD-9 Codes: ['ICD9_4019', 'ICD9_4280', 'ICD9_42731', 'ICD9_5849', 'ICD9_25000', 'ICD9_2724', 'ICD9_51881', 'ICD9_496']
Descriptions: ['Unspecified essential hypertension', 'Congestive heart failure, unspecified', 'Atrial fibrillation', 'Acute kidney failure, unspecified', 'Diabetes mellitus without mention of complication, type II or unspecified type', 'Other and unspecified hyperlipidemia', 'Acute respiratory failure', 'Chronic airway obstruction, not elsewhere classified']
All Probabilities: [('ICD9_4019', 0.36734890937805176), ('ICD9_4280', 0.5146132111549377), ('ICD9_42731', 0.32555848360061646), ('ICD9_41401', 0.14263159036636353), ('ICD9_5849', 0.4151045083999634), ('ICD9_25000', 0.2524208426475525), ('ICD9_2724', 0.20791415870189667), ('ICD9_51881', 0.41107288002967834), ('ICD9_5990', 0.19160877168178558), ('ICD9_53081', 0.18285144865512848), ('ICD9_2720', 0.04882490634918213), ('ICD9_V053', 0.0010593518381938338), ('ICD9_V290', 0.0008660201565362513), ('ICD9_2859', 0

In [ ]:
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

('./longformer_icd9_defintion/tokenizer_config.json',
 './longformer_icd9_defintion/special_tokens_map.json',
 './longformer_icd9_defintion/vocab.json',
 './longformer_icd9_defintion/merges.txt',
 './longformer_icd9_defintion/added_tokens.json')